# Task1

1. Create Delta Tables Using 3 Methods

In [2]:
#Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/Sales1_data.csv", "dbfs:/FileStore/Sales1_data.csv")
#Load CSV data into a DataFrame
df_sales = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/Sales1_data.csv")
#Write DataFrame to Delta format
df_sales.write.format("delta").mode("overwrite").save("/delta/sales1_data")

NameError: name 'dbutils' is not defined

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

# Define schema for customer_data.json file
schema = StructType([ 
                   StructField("CustomerID", StringType(), True), 
                   StructField("CustomerName", StringType(), True), 
                   StructField("Region", StringType(), True), 
                   StructField("SignupDate", StringType(), True)  # or DateType if dates are in valid format
])


In [0]:
# Move the file from Workspace to DBFS
dbutils.fs.cp("file:/Workspace/Shared/customer_data.json", "dbfs:/FileStore/customer_data.json")

True

In [0]:
df_product=spark.read.format("json").schema(schema).load("dbfs:/FileStore/customer_data.json") 
df_product.show()

+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C002|   Jane Smith| South|2023-02-15|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
+----------+-------------+------+----------+



In [0]:
df_product.write.format("delta").mode("overwrite").save("/delta/customer_data")

In [0]:
# Create a sample DataFrame
sample_data = [("John", 30), ("Jane", 25), ("Sam", 35)]
columns = ["Name", "Age"]
sample_df = spark.createDataFrame(sample_data, columns)

# Write the DataFrame as a Parquet file
sample_df.write.mode("overwrite").parquet("/FileStore/sample_parquet_file.parquet")

In [0]:
# Load the Parquet file
parquet_df = spark.read.parquet("/FileStore/sample_parquet_file.parquet")

# Convert the Parquet file to a Delta Table
parquet_df.write.format("delta").mode("overwrite").save("/delta/sample_parquet_to_delta")

# task 2

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")

True

In [0]:
df_new_sales = spark.read.format("csv").option("header", "true").load("/FileStore/new_sales_data.csv")
df_new_sales.write.format("delta").mode("overwrite").save("/delta/new_sales_data")

In [0]:
df_sales = spark.read.format("delta").load("/delta/sales1_data")
df_new_sales = spark.read.format("delta").load("/delta/new_sales_data")

In [0]:
# Create temporary views for SQL operations with corrected view names
df_sales.createOrReplaceTempView("delta_sales1") 

df_new_sales.createOrReplaceTempView("new_sales")

In [0]:
spark.sql("""
    MERGE INTO delta_sales1 AS target
    USING new_sales AS source
    ON target.OrderID = source.OrderID
    WHEN MATCHED THEN UPDATE SET
        target.Product = source.Product,
        target.CustomerID = source.CustomerID
    WHEN NOT MATCHED THEN INSERT 
    (
        OrderID, OrderDate, CustomerID, Product, Quantity, Price
    ) VALUES (
        source.OrderID, source.OrderDate, source.CustomerID, source.Product,source.Quantity, source.Price
    )
""")



DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("SELECT * FROM delta_sales1").show()

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
+-------+----------+----------+--------+--------+-----+



In [0]:
#Write the employee dataframe to a delta table
df_sales.write.format("delta").mode("overwrite").save("/delta/sales1_data")
#Register the delta table
spark.sql("CREATE TABLE IF NOT EXISTS delta_sales1_table USING DELTA LOCATION '/delta/sales1_data'")

DataFrame[]

Task 3 - Optimize Delta Table

In [0]:
spark.sql("""
	OPTIMIZE delta_sales1_table ZORDER BY CustomerID
""")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

Task 4

In [0]:
#Describe history
spark.sql("DESCRIBE HISTORY delta_sales1_table").show(truncate=False)

+-------+-------------------+----------------+----------------------------------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
spark.sql("""
	VACUUM delta_sales_table RETAIN 168 HOURS
""")

NameError: name 'spark' is not defined

Task 5 - Hands on

In [0]:
# Query historical versions using Time Travel
historical_df = spark.read.format("delta").option("versionAsOf", 1).load("/delta/sales1_data")
historical_df.show()


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
+-------+----------+----------+--------+--------+-----+



In [0]:
# Implement Schema Enforcement
df_sales.write.format("delta").mode("append").option("mergeSchema", "true").save("/delta/sales1_data")
